In [4]:
import numpy as np
import os
import pickle
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sklearn.utils import shuffle
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.compose import ColumnEnsembleClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut

In [ ]:
# TODO: Try LOO CV

In [7]:
anish = []
directory = './data_live/data/anish'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        anish.append(np.load(f))

In [8]:
kevin = []
directory = './data_live/data/kevin'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        kevin.append(np.load(f))

In [9]:
anish = np.array(anish)
kevin = np.array(kevin)

In [10]:
anish.shape

(21, 500, 12)

In [11]:
kevin.shape

(14, 500, 12)

In [ ]:
# First 21 is anish, next 14 are kevin
X = np.concatenate((np.array(anish), np.array(kevin)))
y = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
X, y = shuffle(X, y)
print(X.shape)
print(y.shape)

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(X)

In [ ]:
# Two ways to deal with multivariate: concatenate or ensemble. 
# Conclusion: concatenator scores slightly higher, but is much slower

knn = KNeighborsTimeSeriesClassifier(distance="dtw", n_neighbors=3)
# knn = TimeSeriesForestClassifier(n_estimators=2)
X_con = ColumnConcatenator().fit_transform(X)

scores = []
for train_index, test_index in loo.split(X_con):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_con[train_index], X_con[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)[0]
    scores.append(y_pred == y_test)

In [ ]:
print(np.mean(scores))

In [ ]:
# clf = KNeighborsTimeSeriesClassifier(distance="dtw", n_neighbors=2)
estimators = [(f"knn_{i}", KNeighborsTimeSeriesClassifier(distance="dtw", n_neighbors=2), i) for i in range(12)]
col_ens = ColumnEnsembleClassifier(estimators=estimators)

scores = []
for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    col_ens.fit(X_train, y_train)
    y_pred = col_ens.predict(X_test)
    # print(y_pred[0], y_test[0])
    scores.append(y_pred[0] == y_test[0])

In [ ]:
print(np.mean(scores))

In [ ]:
from sktime.classification.interval_based import TimeSeriesForestClassifier

clf = TimeSeriesForestClassifier(n_estimators=10)
estimators = [(f"tsf_{i}", clf, i) for i in range(12)]
col_ens = ColumnEnsembleClassifier(estimators=estimators)

scores = []
for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    col_ens.fit(X_train, y_train)
    y_pred = col_ens.predict(X_test)[0]
    scores.append(y_pred == y_test)

In [ ]:
print(np.mean(scores))

In [ ]:
# Save model
estimators = [(f"knn_{i}", KNeighborsTimeSeriesClassifier(distance="dtw", n_neighbors=2), i) for i in range(12)]
col_ens = ColumnEnsembleClassifier(estimators=estimators)

# Sanity check, predict on entire set
col_ens.fit(X, y)
y_pred = col_ens.predict(X)
print(y_pred.shape)
sum(y_pred == y)/35

In [ ]:
# save
with open('model.pkl','wb') as f:
    pickle.dump(col_ens, f)

In [5]:
# load
with open('model.pkl', 'rb') as f:
    clf2 = pickle.load(f)

In [16]:
anish[0].shape

(500, 12)

In [26]:
clf2.predict(np.expand_dims(anish[0], 0))

array([1])